# Alineamientos de secuencias con Biopython

Autores: **Raúl Mendoza**, **Adrián Ojeda**  
Asignatura: **Bioinformática (ULPGC)**

En esta práctica trabajamos con alineamientos de secuencias de ADN y proteínas.
Vamos a ir resolviendo los tres ejercicios propuestos en la sesión 03 de laboratorio,
comentando el código como si estuviésemos escribiendo nuestras propias notas
para estudiar para el examen.

## Ejercicio 1: Implementación de algoritmos de alineamiento global

En este primer ejercicio nos piden:

1. Programar en Python el algoritmo de **Needleman–Wunsch** para hacer un alineamiento global.
2. Proponer **nuestro propio algoritmo de alineamiento** y también programarlo.
3. Probar ambos algoritmos con varias secuencias cortas (de longitud 10 o menos) para ver los resultados.

Primero recordamos la idea básica de Needleman–Wunsch:

* Se construye una matriz de puntuaciones donde las filas corresponden a la primera secuencia
  y las columnas a la segunda.
* Se inicializa la primera fila y la primera columna con penalizaciones de hueco.
* Cada celda `M[i,j]` se calcula como el máximo de tres opciones:
  - Diagonal: `M[i-1, j-1] + score(coincidencia/desemparejamiento)`
  - Arriba:   `M[i-1, j] + penalización_hueco`
  - Izquierda:`M[i, j-1] + penalización_hueco`
* Al final, desde la esquina inferior derecha se hace un **retroceso (traceback)** para reconstruir el
  alineamiento óptimo.


In [1]:
def needleman_wunsch(seq1, seq2, match_score=1, mismatch_score=-1, gap_score=-1):
    """Implementación sencilla de Needleman–Wunsch para alineamiento global.

    Devuelve las secuencias alineadas y la puntuación final.
    """
    n = len(seq1)
    m = len(seq2)

    # Matriz de puntuaciones
    score = [[0] * (m + 1) for _ in range(n + 1)]

    # Inicialización de la primera fila y primera columna con huecos
    for i in range(1, n + 1):
        score[i][0] = score[i - 1][0] + gap_score
    for j in range(1, m + 1):
        score[0][j] = score[0][j - 1] + gap_score

    # Rellenar la matriz
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if seq1[i - 1] == seq2[j - 1]:
                diag = score[i - 1][j - 1] + match_score
            else:
                diag = score[i - 1][j - 1] + mismatch_score

            up = score[i - 1][j] + gap_score
            left = score[i][j - 1] + gap_score

            score[i][j] = max(diag, up, left)

    # Traceback desde la esquina inferior derecha
    align1 = []
    align2 = []
    i, j = n, m

    while i > 0 or j > 0:
        current = score[i][j]

        # Caso 1: venimos de la diagonal
        if i > 0 and j > 0:
            if seq1[i - 1] == seq2[j - 1]:
                diag = score[i - 1][j - 1] + match_score
            else:
                diag = score[i - 1][j - 1] + mismatch_score
        else:
            diag = float('-inf')

        # Caso 2: venimos de arriba (hueco en seq2)
        if i > 0:
            up = score[i - 1][j] + gap_score
        else:
            up = float('-inf')

        # Caso 3: venimos de la izquierda (hueco en seq1)
        if j > 0:
            left = score[i][j - 1] + gap_score
        else:
            left = float('-inf')

        # Comprobamos de dónde es más coherente venir
        if current == diag:
            align1.append(seq1[i - 1])
            align2.append(seq2[j - 1])
            i -= 1
            j -= 1
        elif current == up:
            align1.append(seq1[i - 1])
            align2.append('-')
            i -= 1
        else:  # current == left
            align1.append('-')
            align2.append(seq2[j - 1])
            j -= 1

    # Damos la vuelta a las listas porque las hemos ido construyendo al revés
    align1 = ''.join(reversed(align1))
    align2 = ''.join(reversed(align2))

    return align1, align2, score[n][m]

# Probamos Needleman–Wunsch con las secuencias del ejemplo de las diapositivas
seq1 = "TCAGACGATTG"
seq2 = "TCGGAGCTG"

al1, al2, puntuacion = needleman_wunsch(seq1, seq2, match_score=5, mismatch_score=-4, gap_score=-6)

print("Secuencia 1:", seq1)
print("Secuencia 2:", seq2)
print("Alineamiento (Needleman–Wunsch):")
print(al1)
print(al2)
print("Puntuación final:", puntuacion)


Secuencia 1: TCAGACGATTG
Secuencia 2: TCGGAGCTG
Alineamiento (Needleman–Wunsch):
TCAGACGATTG
TCGGA-G-CTG
Puntuación final: 15


Con este código ya tenemos una implementación básica de Needleman–Wunsch.
La puntuación final depende de los valores que demos a `match_score`, `mismatch_score`
y `gap_score`. En las diapositivas se usan valores parecidos (bonificando bastante las
coincidencias y penalizando fuerte los huecos).

### 1.b) Nuestro propio algoritmo de alineado

Para la segunda parte vamos a proponer un algoritmo muy sencillo basado en **desplazar**
la secuencia corta sobre la larga sin permitir huecos internos. Solo añadimos huecos al
principio o al final para representar el desplazamiento.

La idea sería:

1. Identificar cuál de las dos secuencias es la más corta.
2. Probar todos los desplazamientos posibles de la corta sobre la larga.
3. Para cada desplazamiento, calcular una puntuación muy simple:
   * +1 por coincidencia
   * -1 por desemparejamiento
4. Nos quedamos con el desplazamiento que tenga mayor puntuación.

Este algoritmo es mucho más limitado que Needleman–Wunsch porque **no permite insertar
huecos intermedios**, pero sirve como ejemplo de otro enfoque más "greedy" y rápido.

In [2]:
def alineamiento_por_desplazamiento(seq1, seq2, match_score=1, mismatch_score=-1):
    """Alineamiento muy simple por desplazamiento de la secuencia corta.

    Solo permite huecos al principio o al final (no hay indels internos).
    """
    # Queremos que s1 sea la larga y s2 la corta para simplificar
    if len(seq1) >= len(seq2):
        larga, corta = seq1, seq2
        swapped = False
    else:
        larga, corta = seq2, seq1
        swapped = True

    L = len(larga)
    S = len(corta)

    mejor_score = float('-inf')
    mejor_desplazamiento = 0

    # Probamos todos los desplazamientos posibles (incluyendo a la izquierda y derecha)
    for offset in range(0, L - S + 1):
        # offset indica dónde empieza la corta respecto a la larga
        score = 0
        for i in range(S):
            if larga[offset + i] == corta[i]:
                score += match_score
            else:
                score += mismatch_score

        if score > mejor_score:
            mejor_score = score
            mejor_desplazamiento = offset

    # Reconstruimos el alineamiento en forma de cadenas con huecos solo fuera de la zona solapada
    prefijos = '-' * mejor_desplazamiento
    sufijos = '-' * (L - (mejor_desplazamiento + S))

    alineada_corta = prefijos + corta + sufijos
    alineada_larga = larga

    # Si originalmente la secuencia "corta" era la segunda, tenemos que devolver en el orden original
    if swapped:
        return alineada_corta, alineada_larga, mejor_score
    else:
        return alineada_larga, alineada_corta, mejor_score

# Probamos nuestro alineador con las mismas secuencias
al1_simple, al2_simple, score_simple = alineamiento_por_desplazamiento(seq1, seq2)
print("Alineamiento por desplazamiento (algoritmo propio):")
print(al1_simple)
print(al2_simple)
print("Puntuación:", score_simple)


Alineamiento por desplazamiento (algoritmo propio):
TCAGACGATTG
TCGGAGCTG--
Puntuación: -1


Vemos que la forma de alinear es distinta: como no aceptamos huecos internos, el
alineamiento se reduce a desplazar la secuencia más corta y aceptar que muchas
posiciones pueden quedar como desemparejamientos. Es un método barato computacionalmente,
pero pierde bastante información en comparación con Needleman–Wunsch.

### 1.c) Pruebas con varias secuencias globales

Ahora generamos varias parejas de secuencias de ADN aleatorias (longitud ≤ 10) y
comparamos el resultado de Needleman–Wunsch con el del alineamiento por desplazamiento.


In [3]:
import random

def generar_adn_aleatorio(longitud):
    return ''.join(random.choice('ACGT') for _ in range(longitud))

random.seed(42)  # para que siempre salgan los mismos ejemplos

for k in range(5):
    l1 = random.randint(5, 10)
    l2 = random.randint(5, 10)
    s1 = generar_adn_aleatorio(l1)
    s2 = generar_adn_aleatorio(l2)

    print("\nEjemplo", k + 1)
    print("Secuencia 1:", s1)
    print("Secuencia 2:", s2)

    nwal1, nwal2, nw_score = needleman_wunsch(s1, s2)
    print("Needleman–Wunsch:")
    print(nwal1)
    print(nwal2)
    print("Puntuación:", nw_score)

    desl1, desl2, sd_score = alineamiento_por_desplazamiento(s1, s2)
    print("Alineamiento por desplazamiento:")
    print(desl1)
    print(desl2)
    print("Puntuación:", sd_score)



Ejemplo 1
Secuencia 1: AGCCCAATAA
Secuencia 2: ACCAC
Needleman–Wunsch:
AGCCCAATAA
A--CC---AC
Puntuación: -2
Alineamiento por desplazamiento:
AGCCCAATAA
ACCAC-----
Puntuación: 1

Ejemplo 2
Secuencia 1: TCTGACTGGC
Secuencia 2: CGAATAGGGA
Needleman–Wunsch:
TCTGACT--GGC
-C-GAATAGGGA
Puntuación: 0
Alineamiento por desplazamiento:
TCTGACTGGC
CGAATAGGGA
Puntuación: -6

Ejemplo 3
Secuencia 1: ATAGGCAACG
Secuencia 2: ACATGTGC
Needleman–Wunsch:
ATAGGCAACG
ACATG-TGC-
Puntuación: -2
Alineamiento por desplazamiento:
ATAGGCAACG
ACATGTGC--
Puntuación: -2

Ejemplo 4
Secuencia 1: CGACCCT
Secuencia 2: TGCGACA
Needleman–Wunsch:
--CGACCCT
TGCGA--CA
Puntuación: -1
Alineamiento por desplazamiento:
CGACCCT
TGCGACA
Puntuación: -3

Ejemplo 5
Secuencia 1: GACGCTT
Secuencia 2: TCGCCGTT
Needleman–Wunsch:
--GACGCTT
TCGCCG-TT
Puntuación: 1
Alineamiento por desplazamiento:
-GACGCTT
TCGCCGTT
Puntuación: -1


En general, Needleman–Wunsch suele conseguir alineamientos más razonables cuando
hay inserciones o deleciones, ya que permite introducir huecos en cualquier punto.
Nuestro alineador simple solo desplaza una secuencia respecto a la otra, por lo que
su calidad baja en cuanto las secuencias empiezan a divergir un poco.

## Ejercicio 2: PairwiseAligner con secuencias de ADN

En este ejercicio tenemos que usar la clase `PairwiseAligner` de **Biopython** para
alinear secuencias de ADN y obtener la puntuación y el alineamiento óptimo en dos
escenarios:

a) Secuencias generadas aleatoriamente.
b) Secuencias obtenidas a partir de ficheros descargados de bases de datos biológicas.

Además, hay que jugar con distintos parámetros: esquema de puntuación, penalizaciones
de huecos y modalidad de alineamiento (global o local) y comentar las diferencias.

In [4]:
import sys

try:
    from Bio import Align
    from Bio.Align import substitution_matrices
except ImportError:
    !pip install biopython -q
    from Bio import Align
    from Bio.Align import substitution_matrices

print("Versión de Python:", sys.version)
print("Biopython importado correctamente.")


Versión de Python: 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]
Biopython importado correctamente.


### 2.a) Alineamiento de secuencias de ADN aleatorias

Primero repetimos la idea de generar secuencias de ADN aleatorias, pero ahora usando
`PairwiseAligner`. Probamos con alineamiento global por defecto y vemos la puntuación
y el alineamiento óptimo que nos devuelve Biopython.

In [5]:
from Bio import Align

aligner = Align.PairwiseAligner()
aligner.mode = 'global'  # alineamiento global

print(aligner)

seq_adn1 = generar_adn_aleatorio(10)
seq_adn2 = generar_adn_aleatorio(8)

print("Secuencia ADN 1:", seq_adn1)
print("Secuencia ADN 2:", seq_adn2)

alignments = aligner.align(seq_adn1, seq_adn2)
best_alignment = alignments[0]

print("\nMejor alineamiento (global):")
print(best_alignment)
print("Puntuación:", best_alignment.score)


Pairwise sequence aligner with parameters
  wildcard: None
  match_score: 1.000000
  mismatch_score: 0.000000
  target_internal_open_gap_score: 0.000000
  target_internal_extend_gap_score: 0.000000
  target_left_open_gap_score: 0.000000
  target_left_extend_gap_score: 0.000000
  target_right_open_gap_score: 0.000000
  target_right_extend_gap_score: 0.000000
  query_internal_open_gap_score: 0.000000
  query_internal_extend_gap_score: 0.000000
  query_left_open_gap_score: 0.000000
  query_left_extend_gap_score: 0.000000
  query_right_open_gap_score: 0.000000
  query_right_extend_gap_score: 0.000000
  mode: global

Secuencia ADN 1: GCCTAAACCT
Secuencia ADN 2: ATTTGAAG

Mejor alineamiento (global):
target            0 GCC-T---AAACCT- 10
                  0 ----|---||----- 15
query             0 ---ATTTGAA----G  8

Puntuación: 3.0


El objeto `PairwiseAligner` ya implementa internamente algoritmos clásicos como
Needleman–Wunsch o Gotoh, dependiendo de cómo configuremos las penalizaciones de huecos.
El formato que imprime Biopython incluye las secuencias alineadas, la posición de inicio
y fin y la puntuación total.

### 2.b) Secuencias de ADN obtenidas de bases de datos biológicas

En un entorno real, las secuencias de ADN se podrían descargar por ejemplo desde **NCBI**
(GenBank) o **Ensembl**. Normalmente se bajan como ficheros en formato **FASTA** y luego
se leen desde Python.

En este cuaderno, para simplificar y no depender de conexión a Internet, vamos a simular
que ya tenemos dos fragmentos de ADN guardados en ficheros FASTA. Son secuencias cortas
para poder ver bien el alineamiento.


In [6]:
# Simulamos lectura de dos secuencias de un fichero FASTA
fasta_seq1 = ">ejemplo_ncbi_1\n" \
             "ATGCGTACGTTAGCCTAG"

fasta_seq2 = ">ejemplo_ncbi_2\n" \
             "ATGACGACGTTGGCCTAA"

def leer_fasta_simple(texto_fasta):
    """Función muy simple para extraer la secuencia de un texto FASTA."""
    lineas = texto_fasta.strip().split("\n")
    secuencias = [l.strip() for l in lineas if not l.startswith('>')]
    return ''.join(secuencias)

seq_db1 = leer_fasta_simple(fasta_seq1)
seq_db2 = leer_fasta_simple(fasta_seq2)

print("Secuencia DB 1:", seq_db1)
print("Secuencia DB 2:", seq_db2)

alignments_db = aligner.align(seq_db1, seq_db2)
best_db = alignments_db[0]

print("\nAlineamiento global con parámetros por defecto:")
print(best_db)
print("Puntuación:", best_db.score)


Secuencia DB 1: ATGCGTACGTTAGCCTAG
Secuencia DB 2: ATGACGACGTTGGCCTAA

Alineamiento global con parámetros por defecto:
target            0 ATG-CGTACGTTAG-CCTAG- 18
                  0 |||-||-|||||-|-||||-- 21
query             0 ATGACG-ACGTT-GGCCTA-A 18

Puntuación: 15.0


Con este ejemplo estaríamos simulando el flujo típico:
1. Descargamos un archivo FASTA de una base de datos.
2. Lo leemos desde Python (aquí hemos usado una función casera muy simple).
3. Alineamos las secuencias con `PairwiseAligner`.

Ahora vamos a cambiar los parámetros de puntuación y el modo de alineamiento para ver
cómo varían los resultados.

### Cambiando parámetros del alineador (puntuaciones y modo global/local)

Ajustamos la puntuación de coincidencias, de desemparejamientos y las penalizaciones de
huecos. Además probamos el modo **local**, que intenta encontrar solo el mejor fragmento
alineado dentro de las secuencias.

In [7]:
# Configuración 1: alineamiento global con huecos muy penalizados
aligner.mode = 'global'
aligner.match_score = 2
aligner.mismatch_score = -1
aligner.open_gap_score = -5
aligner.extend_gap_score = -1

print("Configuración 1 (global, huecos muy caros):")
print(aligner)

alignments_db1 = aligner.align(seq_db1, seq_db2)
print("Mejor alineamiento (configuración 1):")
print(alignments_db1[0])
print("Puntuación:", alignments_db1[0].score)

# Configuración 2: alineamiento local con huecos menos penalizados
aligner.mode = 'local'
aligner.match_score = 2
aligner.mismatch_score = -1
aligner.open_gap_score = -2
aligner.extend_gap_score = -0.5

print("\nConfiguración 2 (local, huecos más baratos):")
print(aligner)

alignments_db2 = aligner.align(seq_db1, seq_db2)
print("Mejor alineamiento (configuración 2):")
print(alignments_db2[0])
print("Puntuación:", alignments_db2[0].score)


Configuración 1 (global, huecos muy caros):
Pairwise sequence aligner with parameters
  wildcard: None
  match_score: 2.000000
  mismatch_score: -1.000000
  target_internal_open_gap_score: -5.000000
  target_internal_extend_gap_score: -1.000000
  target_left_open_gap_score: -5.000000
  target_left_extend_gap_score: -1.000000
  target_right_open_gap_score: -5.000000
  target_right_extend_gap_score: -1.000000
  query_internal_open_gap_score: -5.000000
  query_internal_extend_gap_score: -1.000000
  query_left_open_gap_score: -5.000000
  query_left_extend_gap_score: -1.000000
  query_right_open_gap_score: -5.000000
  query_right_extend_gap_score: -1.000000
  mode: global

Mejor alineamiento (configuración 1):
target            0 ATGCGTACGTTAGCCTAG 18
                  0 |||...|||||.|||||. 18
query             0 ATGACGACGTTGGCCTAA 18

Puntuación: 21.0

Configuración 2 (local, huecos más baratos):
Pairwise sequence aligner with parameters
  wildcard: None
  match_score: 2.000000
  mismatch_s

En el modo **global** se intenta alinear las secuencias de principio a fin, aunque eso
implique introducir huecos. Al subir mucho la penalización de huecos, el alineador tiende
a evitar abrir nuevos huecos y prefiere tolerar más desemparejamientos.

En el modo **local**, en cambio, el alineador se centra en encontrar el tramo con más
similaridad. Muchas veces devuelve un fragmento interno donde la identidad es alta y
"ignora" los extremos si aportan poca información.

## Ejercicio 3: PairwiseAligner con secuencias de aminoácidos

Ahora repetimos la idea anterior, pero con **proteínas** en lugar de ADN. En este caso
las puntuaciones de coincidencia/desemparejamiento suelen venir dadas por una **matriz de
sustitución**, como BLOSUM62 o PAM250.

Biopython trae ya muchas matrices predefinidas y también nos permite usar una matriz
propia.

### 3.a) Secuencias de aminoácidos aleatorias

Generamos secuencias de aminoácidos usando el alfabeto estándar de 20 residuos y las
alineamos con `PairwiseAligner` usando la matriz BLOSUM62.

In [8]:
aminoacidos = "ACDEFGHIKLMNPQRSTVWY"

def generar_proteina_aleatoria(longitud):
    return ''.join(random.choice(aminoacidos) for _ in range(longitud))

prot1 = generar_proteina_aleatoria(12)
prot2 = generar_proteina_aleatoria(10)

print("Proteína 1:", prot1)
print("Proteína 2:", prot2)

aligner_prot = Align.PairwiseAligner()
aligner_prot.mode = 'global'
matrix_blosum62 = substitution_matrices.load('BLOSUM62')
aligner_prot.substitution_matrix = matrix_blosum62

al_prot = aligner_prot.align(prot1, prot2)[0]
print("\nAlineamiento de proteínas (BLOSUM62, global):")
print(al_prot)
print("Puntuación:", al_prot.score)


Proteína 1: MELQGRAKTGTE
Proteína 2: LTYHFNGVTA

Alineamiento de proteínas (BLOSUM62, global):
target            0 MELQGRAKT----G-TE- 12
                  0 --|-----|----|-|-- 18
query             0 --L-----TYHFNGVT-A 10

Puntuación: 20.0


BLOSUM62 favorece sustituciones que se ven con frecuencia en proteínas reales y penaliza
más aquellas que son muy raras evolutivamente. Por eso no todas las no coincidencias
valen lo mismo: cambiar una leucina por isoleucina no se penaliza igual que cambiarla
por una prolina, por ejemplo.

### 3.b) Secuencias de proteínas de bases de datos biológicas

En sitios como **UniProt** o el propio **NCBI** podemos descargar secuencias de proteínas
en formato FASTA. Igual que antes, aquí vamos a simular que ya las tenemos guardadas y
las cargamos desde una cadena de texto.

Luego probaremos distintas matrices de sustitución: **BLOSUM62**, **PAM250** y una matriz
personalizada muy sencilla, para comparar las puntuaciones que obtenemos.

In [9]:
# Simulamos dos fragmentos de proteínas procedentes de UniProt (inventando identificadores)
fasta_prot1 = ">P_EJEMPLO_1\n" \
               "MVLSPADKTNVKAAW"

fasta_prot2 = ">P_EJEMPLO_2\n" \
               "MALSPADKTNIKAAW"

prot_db1 = leer_fasta_simple(fasta_prot1)
prot_db2 = leer_fasta_simple(fasta_prot2)

print("Proteína DB 1:", prot_db1)
print("Proteína DB 2:", prot_db2)

# Función auxiliar para alinear con una matriz concreta
def alinear_con_matriz(seq1, seq2, matrix_name=None, matrix_obj=None, mode='global'):
    a = Align.PairwiseAligner()
    a.mode = mode
    if matrix_obj is None and matrix_name is not None:
        a.substitution_matrix = substitution_matrices.load(matrix_name)
    elif matrix_obj is not None:
        a.substitution_matrix = matrix_obj
    else:
        raise ValueError("Hay que pasar un nombre de matriz o un objeto de matriz")
    alineamiento = a.align(seq1, seq2)[0]
    return alineamiento

# 1) BLOSUM62
al_blosum = alinear_con_matriz(prot_db1, prot_db2, matrix_name='BLOSUM62')
print("\nAlineamiento con BLOSUM62:")
print(al_blosum)
print("Puntuación BLOSUM62:", al_blosum.score)

# 2) PAM250
al_pam = alinear_con_matriz(prot_db1, prot_db2, matrix_name='PAM250')
print("\nAlineamiento con PAM250:")
print(al_pam)
print("Puntuación PAM250:", al_pam.score)

# 3) Matriz personalizada muy simple: +2 match, -1 mismatch para un subconjunto de aminoácidos
from Bio.Align import substitution_matrices

letras = ''.join(sorted(set(prot_db1 + prot_db2)))  # solo las letras que aparecen en estas proteínas
# IMPORTANTE: indicar dims=2 para crear una matriz cuadrada (2D)
custom = substitution_matrices.Array(alphabet=letras, dims=2)
for aa1 in letras:
    for aa2 in letras:
        if aa1 == aa2:
            custom[aa1, aa2] = 2
        else:
            custom[aa1, aa2] = -1

al_custom = alinear_con_matriz(prot_db1, prot_db2, matrix_obj=custom)
print("\nAlineamiento con matriz personalizada:")
print(al_custom)
print("Puntuación matriz personalizada:", al_custom.score)


Proteína DB 1: MVLSPADKTNVKAAW
Proteína DB 2: MALSPADKTNIKAAW

Alineamiento con BLOSUM62:
target            0 MV-LSPADKTNVKAAW 15
                  0 |--||||||||.|||| 16
query             0 M-ALSPADKTNIKAAW 15

Puntuación BLOSUM62: 73.0

Alineamiento con PAM250:
target            0 MV-LSPADKTNVKAAW 15
                  0 |--||||||||.|||| 16
query             0 M-ALSPADKTNIKAAW 15

Puntuación PAM250: 66.0

Alineamiento con matriz personalizada:
target            0 MV-LSPADKTNV-KAAW 15
                  0 |--||||||||--|||| 17
query             0 M-ALSPADKTN-IKAAW 15

Puntuación matriz personalizada: 26.0


Las tres matrices dan **alineamientos muy parecidos**, porque las secuencias son casi
idénticas (solo cambian un par de residuos). Sin embargo, la puntuación sí puede variar:

* Con **BLOSUM62** y **PAM250** las sustituciones se ponderan según cuánto se observan en
  secuencias homólogas reales.
* Con la **matriz personalizada** todas las coincidencias valen lo mismo y todas las
  no coincidencias también, así que perdemos matices evolutivos.

En problemas reales de bioinformática suele ser importante elegir una matriz adecuada al
nivel de divergencia esperado entre las proteínas: PAM250, por ejemplo, está más pensada
para secuencias relativamente alejadas, mientras que BLOSUM62 funciona bien en muchos
casos prácticos de identidad media.

---

Con esto damos por resueltos los tres ejercicios de la sesión de laboratorio. En el
cuaderno hemos implementado un algoritmo clásico (Needleman–Wunsch), un alineador
alternativo sencillo y varios ejemplos usando `PairwiseAligner` con ADN y proteínas,
incluyendo el uso de matrices de sustitución como BLOSUM62 y PAM250.

Raúl Mendoza  
Adrián Ojeda
